In [ ]:
import pandas as pd 
import numpy as np
import pandasql as ps
import os
import random 
import numpy as np
import warnings
import tensorflow as tf 
from preprocess_days_stats import preprocess_match_days
from preprocess_time_serie import preprocess_teams, preprocess_time_series_features
from tensorflow.keras import layers
from helper_functions_tensorflow import CSVLoggerCallback, CSVLoggerCallbackParams, plot_loss_curve
from imblearn.over_sampling import RandomOverSampler

# Ignora tutti i warning temporaneamente
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

df_giornate = preprocess_match_days(r"c:\Users\Hp\Documents\Serie_A_dump\csv_serie_a")

In [4]:
df_giornate.head()

,div,giornata,stagione,date,hometeam,awayteam,ft_home_goals,ft_away_goals,ft_result,ht_home_goals,ht_away_goals,ht_results,home_shots,away_shots,home_shots_targ,away_shots_targ,home_fouls,away_fouls,home_corners,away_corners,home_yellow,away_yellow,home_red,away_red,home_win_odds,draw_odds,away_win_odds
0,I1,1,2005/2006,2005-08-27,Fiorentina,Sampdoria,2.0,1.0,H,2.0,0.0,H,15.0,9.0,9.0,4.0,18.0,23.0,9.0,3.0,1.0,4.0,0.0,0.0,2.20,2.87,3.25
1,I1,1,2005/2006,2005-08-27,Livorno,Lecce,2.0,1.0,H,1.0,1.0,D,17.0,6.0,7.0,5.0,27.0,21.0,9.0,2.0,2.0,3.0,0.0,0.0,1.90,2.90,4.00
2,I1,1,2005/2006,2005-08-28,Ascoli,Milan,1.0,1.0,D,0.0,0.0,D,8.0,16.0,3.0,9.0,22.0,16.0,0.0,5.0,2.0,1.0,0.0,0.0,7.50,3.75,1.40
3,I1,1,2005/2006,2005-08-28,Inter,Treviso,3.0,0.0,H,1.0,0.0,H,16.0,7.0,9.0,3.0,13.0,20.0,6.0,1.0,1.0,3.0,0.0,0.0,1.16,5.50,13.00
4,I1,1,2005/2006,2005-08-28,Juventus,Chievo,1.0,0.0,H,1.0,0.0,H,16.0,2.0,7.0,0.0,16.0,12.0,6.0,2.0,1.0,2.0,0.0,0.0,1.25,5.00,13.00


In [77]:
all_features = ['ft_goals','ft_goals_conceded','shots','shots_target', 'fouls_done','corners_obtained', 'yellows', 'reds']
less_features = ['ft_goals','ft_goals_conceded','shots', 'fouls_done','corners_obtained', 'reds']
few_features = ['ft_goals','ft_goals_conceded','shots', 'reds']

In [76]:
Statistiche_squadre_dict = preprocess_teams(dataframe = df_giornate)
df_Serie_A = preprocess_time_series_features('all', Statistiche_squadre_dict, df_giornate, 3)
df_Serie_A[(df_Serie_A['hometeam']=='Fiorentina') | (df_Serie_A['awayteam']=='Fiorentina')].head(6)

utilizzando tutte le features
preprocess finished


,div,giornata,stagione,date,hometeam,awayteam,home_ft_goals_1,home_ft_goals_2,home_ft_goals_3,home_ft_goals_conceded_1,home_ft_goals_conceded_2,home_ft_goals_conceded_3,home_shots_1,home_shots_2,home_shots_3,home_shots_target_1,home_shots_target_2,home_shots_target_3,home_fouls_done_1,home_fouls_done_2,home_fouls_done_3,home_corners_obtained_1,home_corners_obtained_2,home_corners_obtained_3,home_yellows_1,home_yellows_2,home_yellows_3,home_reds_1,home_reds_2,home_reds_3,away_ft_goals_1,away_ft_goals_2,away_ft_goals_3,away_ft_goals_conceded_1,away_ft_goals_conceded_2,away_ft_goals_conceded_3,away_shots_1,away_shots_2,away_shots_3,away_shots_target_1,away_shots_target_2,away_shots_target_3,away_fouls_done_1,away_fouls_done_2,away_fouls_done_3,away_corners_obtained_1,away_corners_obtained_2,away_corners_obtained_3,away_yellows_1,away_yellows_2,away_yellows_3,away_reds_1,away_reds_2,away_reds_3,ft_result,home_win_odds,draw_odds,away_win_odds
0,I1,1,2005/2006,2005-08-27,Fiorentina,Sampdoria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H,2.20,2.87,3.25
16,I1,2,2005/2006,2005-09-11,Messina,Fiorentina,0.0,NaN,NaN,1.0,NaN,NaN,16.0,NaN,NaN,7.0,NaN,NaN,20.0,NaN,NaN,10.0,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN,15.0,NaN,NaN,9.0,NaN,NaN,18.0,NaN,NaN,9.0,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,D,2.50,2.87,2.75
23,I1,3,2005/2006,2005-09-18,Fiorentina,Udinese,2.0,2.0,NaN,2.0,1.0,NaN,12.0,15.0,NaN,7.0,9.0,NaN,20.0,18.0,NaN,7.0,9.0,NaN,3.0,1.0,NaN,0.0,0.0,NaN,1.0,1.0,NaN,0.0,0.0,NaN,10.0,12.0,NaN,5.0,8.0,NaN,20.0,20.0,NaN,4.0,6.0,NaN,4.0,2.0,NaN,0.0,0.0,NaN,H,2.20,3.00,3.40
33,I1,4,2005/2006,2005-09-21,Lecce,Fiorentina,0.0,0.0,1.0,3.0,0.0,2.0,7.0,9.0,6.0,5.0,6.0,5.0,19.0,18.0,21.0,3.0,9.0,2.0,1.0,2.0,3.0,0.0,0.0,0.0,4.0,2.0,2.0,2.0,2.0,1.0,11.0,12.0,15.0,6.0,7.0,9.0,24.0,20.0,18.0,4.0,7.0,9.0,0.0,3.0,1.0,0.0,0.0,0.0,A,3.75,3.00,2.10
44,I1,5,2005/2006,2005-09-25,Inter,Fiorentina,1.0,3.0,2.0,0.0,0.0,3.0,17.0,12.0,15.0,10.0,8.0,6.0,18.0,16.0,14.0,7.0,5.0,6.0,2.0,1.0,1.0,0.0,0.0,0.0,3.0,4.0,2.0,1.0,2.0,2.0,14.0,11.0,12.0,9.0,6.0,7.0,16.0,24.0,20.0,6.0,4.0,7.0,1.0,0.0,3.0,0.0,0.0,0.0,H,1.61,3.40,6.00
53,I1,6,2005/2006,2005-10-02,Fiorentina,Livorno,0.0,3.0,4.0,1.0,1.0,2.0,4.0,14.0,11.0,2.0,9.0,6.0,20.0,16.0,24.0,3.0,6.0,4.0,3.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,9.0,5.0,6.0,2.0,3.0,4.0,10.0,13.0,17.0,4.0,1.0,5.0,0.0,0.0,2.0,0.0,0.0,0.0,H,1.66,3.40,5.50
